# Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import zipfile as zf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.model_selection import PredefinedSplit

from tqdm.notebook import tqdm

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import pickle

from sklearn.metrics import f1_score

# Загрузка и очистка данныx

In [ ]:
file = zf.ZipFile('/content/drive/MyDrive/data science/ML/Самостоятельная работа ML/data_4class.zip')
file.extractall()

In [ ]:
df = pd.read_csv('/content/data_4class.csv', sep=';')
df

,x1,x2,x3,x4,x5,x6,class
0,"4,515297646252625","-2,07036706615714","3,2585383730838378","3,7930594045972192","3,4720484621322356","-0,4789201770813123",3
1,"1,5508794670649269","1,3820146712061634","1,942693485886148","2,069885683865084","-0,22511193683695682","-2,303949682856058",2
2,"2,9880972406280066","-2,1365756342215176","2,469954527100078","1,7252063508758542","2,971663586773672","-2,543173242953806",3
3,"3,139767963592046","-1,677145546231046","4,138218458647584","2,521013012549771","2,650713586139205","-0,3626219014045815",3
4,"3,1197797829928464","-2,10074721921465","3,699075357830096","2,0461730318657594","2,9990916631660087","-1,6752961527892165",3
...,...,...,...,...,...,...,...
79995,"1,4322293723901158","2,577585066292325","2,9289348771877126","2,469154608484738","-1,0892062411168344","-3,0116797224659697",3
79996,"5,542870566778231","-4,350409600487184","0,3511813929470158","2,6238820108051475","5,749305074824563","-6,686529946039072",3
79997,"2,63199942217274","-1,7480177028185402","3,0505605609724165","1,7208149055436084","2,514794267502953","-1,518916856679832",3
79998,"3,982406251164055","2,620576354922133","-2,7709626535506384","5,519990383556996",NaN,"-1,3776815330357355",2


In [ ]:
df.duplicated().sum() # проверка на дубликаты

0

In [ ]:
df.isna().sum().sum() # проверкаа на пропуски

27158

In [ ]:
df = df.dropna() # удаление пропусков
print(df.isna().sum().sum())
df

0


,x1,x2,x3,x4,x5,x6,class
0,"4,515297646252625","-2,07036706615714","3,2585383730838378","3,7930594045972192","3,4720484621322356","-0,4789201770813123",3
1,"1,5508794670649269","1,3820146712061634","1,942693485886148","2,069885683865084","-0,22511193683695682","-2,303949682856058",2
2,"2,9880972406280066","-2,1365756342215176","2,469954527100078","1,7252063508758542","2,971663586773672","-2,543173242953806",3
3,"3,139767963592046","-1,677145546231046","4,138218458647584","2,521013012549771","2,650713586139205","-0,3626219014045815",3
4,"3,1197797829928464","-2,10074721921465","3,699075357830096","2,0461730318657594","2,9990916631660087","-1,6752961527892165",3
...,...,...,...,...,...,...,...
79994,"-3,6889522034691513","-2,03030768541395","5,561773755122672","-5,125655762226804","0,19355716473086346","-0,20490687574481736",0
79995,"1,4322293723901158","2,577585066292325","2,9289348771877126","2,469154608484738","-1,0892062411168344","-3,0116797224659697",3
79996,"5,542870566778231","-4,350409600487184","0,3511813929470158","2,6238820108051475","5,749305074824563","-6,686529946039072",3
79997,"2,63199942217274","-1,7480177028185402","3,0505605609724165","1,7208149055436084","2,514794267502953","-1,518916856679832",3


# Предобработка данных и выделение значимых атрибутов

In [ ]:
df = df.reindex(range(len(df)), method='ffill')
df

,x1,x2,x3,x4,x5,x6,class
0,"4,515297646252625","-2,07036706615714","3,2585383730838378","3,7930594045972192","3,4720484621322356","-0,4789201770813123",3
1,"1,5508794670649269","1,3820146712061634","1,942693485886148","2,069885683865084","-0,22511193683695682","-2,303949682856058",2
2,"2,9880972406280066","-2,1365756342215176","2,469954527100078","1,7252063508758542","2,971663586773672","-2,543173242953806",3
3,"3,139767963592046","-1,677145546231046","4,138218458647584","2,521013012549771","2,650713586139205","-0,3626219014045815",3
4,"3,1197797829928464","-2,10074721921465","3,699075357830096","2,0461730318657594","2,9990916631660087","-1,6752961527892165",3
...,...,...,...,...,...,...,...
56427,"1,654091590850613","2,3052018415206277","2,9783290888204585","2,6311445602646035","-0,8146397462591817","-2,633101200590537",0
56428,"1,654091590850613","2,3052018415206277","2,9783290888204585","2,6311445602646035","-0,8146397462591817","-2,633101200590537",0
56429,"5,22961597904812","-2,507693395962547","2,794338386098837","4,217209458413095","4,099407546039043","-1,1976450237215426",3
56430,"4,556475532469856","-3,417744660051489","1,5635614553796477","2,427510189563869","4,604257637400534","-4,463168098990028",1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56432 entries, 0 to 56431
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   x1      56432 non-null  object
 1   x2      56432 non-null  object
 2   x3      56432 non-null  object
 3   x4      56432 non-null  object
 4   x5      56432 non-null  object
 5   x6      56432 non-null  object
 6   class   56432 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 3.0+ MB


In [ ]:
# подготовка к переводу во float
df.x1 = df.x1.str.replace(",", ".")
df.x2 = df.x2.str.replace(",", ".")
df.x3 = df.x3.str.replace(",", ".")
df.x4 = df.x4.str.replace(",", ".")
df.x5 = df.x5.str.replace(",", ".")
df.x6 = df.x6.str.replace(",", ".")
df

,x1,x2,x3,x4,x5,x6,class
0,4.515297646252625,-2.07036706615714,3.2585383730838378,3.7930594045972192,3.4720484621322356,-0.4789201770813123,3
1,1.5508794670649269,1.3820146712061634,1.942693485886148,2.069885683865084,-0.22511193683695682,-2.303949682856058,2
2,2.9880972406280066,-2.1365756342215176,2.469954527100078,1.7252063508758542,2.971663586773672,-2.543173242953806,3
3,3.139767963592046,-1.677145546231046,4.138218458647584,2.521013012549771,2.650713586139205,-0.3626219014045815,3
4,3.1197797829928464,-2.10074721921465,3.699075357830096,2.0461730318657594,2.9990916631660087,-1.6752961527892165,3
...,...,...,...,...,...,...,...
56427,1.654091590850613,2.3052018415206277,2.9783290888204585,2.6311445602646035,-0.8146397462591817,-2.633101200590537,0
56428,1.654091590850613,2.3052018415206277,2.9783290888204585,2.6311445602646035,-0.8146397462591817,-2.633101200590537,0
56429,5.22961597904812,-2.507693395962547,2.794338386098837,4.217209458413095,4.099407546039043,-1.1976450237215426,3
56430,4.556475532469856,-3.417744660051489,1.5635614553796477,2.427510189563869,4.604257637400534,-4.463168098990028,1


In [ ]:
df = df.astype(float) # перевод во float
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56432 entries, 0 to 56431
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      56432 non-null  float64
 1   x2      56432 non-null  float64
 2   x3      56432 non-null  float64
 3   x4      56432 non-null  float64
 4   x5      56432 non-null  float64
 5   x6      56432 non-null  float64
 6   class   56432 non-null  float64
dtypes: float64(7)
memory usage: 3.0 MB


In [ ]:
X = df.drop(['class'], axis=1) # выбор признаков
X

,x1,x2,x3,x4,x5,x6
0,4.515298,-2.070367,3.258538,3.793059,3.472048,-0.478920
1,1.550879,1.382015,1.942693,2.069886,-0.225112,-2.303950
2,2.988097,-2.136576,2.469955,1.725206,2.971664,-2.543173
3,3.139768,-1.677146,4.138218,2.521013,2.650714,-0.362622
4,3.119780,-2.100747,3.699075,2.046173,2.999092,-1.675296
...,...,...,...,...,...,...
56427,1.654092,2.305202,2.978329,2.631145,-0.814640,-2.633101
56428,1.654092,2.305202,2.978329,2.631145,-0.814640,-2.633101
56429,5.229616,-2.507693,2.794338,4.217209,4.099408,-1.197645
56430,4.556476,-3.417745,1.563561,2.427510,4.604258,-4.463168


In [ ]:
y = df['class'].astype(int) # выбор целевого признака
y

0        3
1        2
2        3
3        3
4        3
        ..
56427    0
56428    0
56429    3
56430    1
56431    1
Name: class, Length: 56432, dtype: int64

# Подготовка обучающей и проверочной выборок

In [ ]:
# нормализация данных
scaler = StandardScaler()
X_ss = scaler.fit_transform(X)
X_ss.shape

(56432, 6)

In [ ]:
# добавление полиномиальных фич
poly = PolynomialFeatures(2)
X_ss_poly = poly.fit_transform(X_ss)

print(X_ss_poly.shape)

(56432, 28)


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X_ss, y, test_size=0.1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
# X_train, X_test, y_train, y_test = train_test_split(X_ss_poly, y, test_size=0.1, stratify=y)

# Сравнение различных моделей машинного обучения

## Деревo

In [ ]:
dtc = DecisionTreeClassifier()
dtc = dtc.fit(X_train, y_train)

In [ ]:
# pickle.dump(dtc, "/content/drive/MyDrive/data science/ML/Самостоятельная работа ML/dtc.pkl")

['/content/drive/MyDrive/data science/ML/Самостоятельная работа ML/dtc.sav']

In [ ]:
# dtc = pickle.load("/content/drive/MyDrive/data science/ML/Самостоятельная работа ML/dtc.pkl")

In [ ]:
dtc_pred = dtc.predict(X_test)
dtc_pred

array([3, 2, 0, ..., 1, 1, 1])

In [ ]:
f1_score(y_test, dtc_pred, average="micro")

0.8754429482636428

## Случайный лес

In [ ]:
rfc = RandomForestClassifier()

In [ ]:
# rfc = rfc.fit(X_train, y_train)
# f1_score(y_test, rfc.predict(X_test), average="macro")

In [ ]:
# визуализация валидационной выборки для обучения
ps = PredefinedSplit(test_fold=[-1 if i in X_train.index else 0 for i in X.index])
ps

PredefinedSplit(test_fold=array([-1, -1, ..., -1,  0]))

In [ ]:
rfc_rs = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid={
        'n_estimators': [20, 50, 100, 200, 250, 300],
        'max_depth': [7, 8, 10, 20, 25, None],
        'max_features': ['sqrt', 'log2', None],
        'criterion': ['gini', 'entropy', 'log_loss'],
        'min_samples_split': [1, 2, 5, 10],
        'n_jobs': [-1]
    },
    scoring='f1',
    verbose=52,
    cv=ps
)
# model.fit(X_train, y_train)
rfc_rs.fit(X, y)
f1_score(y_test, rfc_rs.predict(X_test), average='micro')

Fitting 1 folds for each of 720 candidates, totalling 720 fits
[CV 1/1; 1/720] START criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=1, n_estimators=20, n_jobs=-1
[CV 1/1; 1/720] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=1, n_estimators=20, n_jobs=-1;, score=nan total time=   0.0s
[CV 1/1; 2/720] START criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=1, n_estimators=50, n_jobs=-1
[CV 1/1; 2/720] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=1, n_estimators=50, n_jobs=-1;, score=nan total time=   0.0s
[CV 1/1; 3/720] START criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=1, n_estimators=100, n_jobs=-1
[CV 1/1; 3/720] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=1, n_estimators=100, n_jobs=-1;, score=nan total time=   0.0s
[CV 1/1; 4/720] START criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=1, n_estimators=200, n_jobs=-1
[CV 1/1; 4/720] END c

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1146, in f1_score
    return fbeta_score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1287, in fbeta_score
    _, _, f, _ = precision_recall_fscore_support(
  File "/usr/local/l

[CV 1/1; 5/720] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=2, n_estimators=20, n_jobs=-1;, score=nan total time=   1.3s
[CV 1/1; 6/720] START criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=2, n_estimators=50, n_jobs=-1


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1146, in f1_score
    return fbeta_score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1287, in fbeta_score
    _, _, f, _ = precision_recall_fscore_support(
  File "/usr/local/l

[CV 1/1; 6/720] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=2, n_estimators=50, n_jobs=-1;, score=nan total time=   3.1s
[CV 1/1; 7/720] START criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=2, n_estimators=100, n_jobs=-1


...

[CV 1/1; 281/720] END criterion=entropy, max_depth=7, max_features=None, min_samples_split=5, n_estimators=20, n_jobs=-1;, score=nan total time=   5.9s
[CV 1/1; 282/720] START criterion=entropy, max_depth=7, max_features=None, min_samples_split=5, n_estimators=50, n_jobs=-1


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1146, in f1_score
    return fbeta_score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1287, in fbeta_score
    _, _, f, _ = precision_recall_fscore_support(
  File "/usr/local/l

[CV 1/1; 282/720] END criterion=entropy, max_depth=7, max_features=None, min_samples_split=5, n_estimators=50, n_jobs=-1;, score=nan total time=  15.0s
[CV 1/1; 283/720] START criterion=entropy, max_depth=7, max_features=None, min_samples_split=5, n_estimators=100, n_jobs=-1


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1146, in f1_score
    return fbeta_score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1287, in fbeta_score
    _, _, f, _ = precision_recall_fscore_support(
  File "/usr/local/l

[CV 1/1; 283/720] END criterion=entropy, max_depth=7, max_features=None, min_samples_split=5, n_estimators=100, n_jobs=-1;, score=nan total time=  30.2s
[CV 1/1; 284/720] START criterion=entropy, max_depth=7, max_features=None, min_samples_split=5, n_estimators=200, n_jobs=-1


In [ ]:
with open("/content/drive/MyDrive/data science/ML/Самостоятельная работа ML/rfc_rs4.pkl", 'wb') as file:
    pickle.dump(rfc_rs, file)

In [ ]:
# pickle.load("/content/drive/MyDrive/data science/ML/Самостоятельная работа ML/rfc_rs.pkl")

In [ ]:
f1_score(y_test, rfc_rs.predict(X_test), average="micro")

In [ ]:
rfc_rs.best_params_

In [ ]:
rfc_rs_df = pd.DataFrame(rfc_rs.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
rfc_rs_df

## Машины опорных векторов

In [ ]:
svc = SVC() # lin 0,8871 pol 0,8825 rbf 0,8869 sig 0,7654
# svc = svc.fit(X_train,y_train)
# f1_score(y_test, svc.predict(X_test), average="micro")

In [ ]:
parameters = {'C': [15, 20, 25],
              'gamma': ['auto','scale'],
              'kernel': ['linear']}

In [ ]:
svc_gs=GridSearchCV(estimator = svc,
                        param_grid = parameters,
                        scoring='f1',
                        cv=ps)

svc_gs.fit(X_train, y_train)

ValueError: Invalid parameter 'n_jobs' for estimator SVC(C=20, gamma='auto', kernel='linear'). Valid parameters are: ['C', 'break_ties', 'cache_size', 'class_weight', 'coef0', 'decision_function_shape', 'degree', 'gamma', 'kernel', 'max_iter', 'probability', 'random_state', 'shrinking', 'tol', 'verbose'].

In [ ]:
f1_score(y_test, svc_gs.predict(X_test), average="micro")

In [ ]:
svc_gs.best_params_

In [ ]:
svc_gs_df = pd.DataFrame(svc_gs.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
svc_gs_df